In [1]:
import psycopg2 as pg
import pandas as pd
import os
from datetime import date, datetime, timedelta

In [2]:
def pandafy(rows, colnames):
    N=len(colnames)
    D={cn: [] for cn in colnames}
    for row in rows:
        for i in range(N): D[colnames[i]].append(row[i])
    pdfy=pd.DataFrame(D)
    return pdfy

def excuteSingleSQLstatement(sql, host, database, user, password):
    conn = pg.connect(host=host,database=database, user=user, password=password)
    cur = conn.cursor()
    cur.execute(sql)
    cur.close()
    conn.commit()
    conn.close()    

def getSQLfromQuery(sql, params, host, database, user, password):
    conn = pg.connect(host=host, database=database, user=user, password=password)
    cur  = conn.cursor()    
    if len(params)==0:
        cur.execute(sql)
    else: 
        cur.execute(sql, params)
    data = cur.fetchall()
    colnames = [desc[0] for desc in cur.description]
    cur.close()
    conn.close()
    df=pandafy(data, colnames)
    return df

#*********************************************
#this is new and good for many insert queries
#*********************************************
def excuteManySQLstatements(sql_list, host, database, user, password):
    conn = pg.connect(host=host,database=database, user=user, password=password)
    cur = conn.cursor()
    for sql in sql_list: cur.execute(sql)
    cur.close()
    conn.commit()
    conn.close()    

In [3]:
def getQuarter(d):
    #d should be a date object
    if d.month<=3: return "Q1"
    if d.month<=6: return "Q2"
    if d.month<=9: return "Q3"
    return "Q4"

def getSemester(d):
    if d.month<=6: return "S1"
    return "S2"

def getWeekYear(d):
    d_J1=date(d.year,1,1)
    w1= d_J1.toordinal() // 7
    w2= d.toordinal() // 7
    return w2-w1+1

def getWeekMonth(d):
    #controversial...
    d_J1=date(d.year,d.month,1)
    w1= d_J1.toordinal() // 7
    w2= d.toordinal() // 7
    return w2-w1+1

def getWeekDay(d):
    WD={0: "Monday", 1: "Tuesday", 2: "Wednesday",
        3: "Thursday", 4: "Friday", 5: "Saturday", 6: "Sunday"}
    return WD[d.weekday()]

def getDayType(d):
    if d.weekday()>=5: return "Weekend"
    return "Working Day"

# Conexão

In [4]:
database='teste'
password='123'

conn = pg.connect(host='localhost', database = database, user = 'postgres', password = password)
conn.close()

# Company

In [5]:
old_company = pd.read_csv('company_final.csv', index_col=0)
old_company=old_company.sort_values(by='name').reset_index(drop=True)
old_company

,ticker,name
0,AMZN,Amazon
1,AAPL,Apple
2,GOOGL,Google
3,MSFT,Microsoft
4,TSLA,Tesla


+ dataset antigo de company tinha poucos dados, tivemos de arranjar outro dataset com mais informação

In [6]:
#df_company = pd.read_csv('companies_sorted.csv', index_col=0)
#df_company.head(3)

In [7]:
#dimcompany = df_company[((df_company['name']=='apple') & (df_company['domain']=='apple.com'))
#                   |((df_company['name']=='microsoft') & (df_company['domain']=='microsoft.com'))
#                  | ((df_company['name']=='tesla') & (df_company['domain']=='teslamotors.com'))
#                  | ((df_company['name']=='amazon') & (df_company['domain']=='amazon.com'))
#                  | ((df_company['name']=='google') & (df_company['domain']=='google.com'))
#                       ]
#dimcompany=dimcompany.sort_values(by='name').reset_index(drop=True)
#dimcompany

In [8]:
#dimcompany.to_csv('company_dwh.csv', index=False)

In [9]:
dimcompany=pd.read_csv('company_dwh.csv')

+ adicionar informação de ticker, CEO e mercado de ações

In [10]:
dimcompany.insert(loc=0,
          column = 'ticker',
          value = old_company.ticker)
dimcompany['CEO'] = ['Jeff Bezos', 'Tim Cook', 'Sundar Pichai', 'Satya Nadella', 'Elon Musk']
dimcompany.insert(loc=2,
          column = 'stock_market',
          value = ['NASDAQ','NASDAQ','NASDAQ','NASDAQ','NASDAQ'])
dimcompany

,ticker,name,stock_market,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate,CEO
0,AMZN,amazon,NASDAQ,amazon.com,1994.0,internet,10001+,"seattle, washington, united states",united states,linkedin.com/company/amazon,93247,161866,Jeff Bezos
1,AAPL,apple,NASDAQ,apple.com,1976.0,consumer electronics,10001+,"cupertino, california, united states",united states,linkedin.com/company/apple,90095,179531,Tim Cook
2,GOOGL,google,NASDAQ,google.com,1998.0,internet,10001+,"mountain view, california, united states",united states,linkedin.com/company/google,75109,131349,Sundar Pichai
3,MSFT,microsoft,NASDAQ,microsoft.com,1975.0,computer software,10001+,"redmond, washington, united states",united states,linkedin.com/company/microsoft,116196,276983,Satya Nadella
4,TSLA,tesla,NASDAQ,teslamotors.com,2003.0,automotive,10001+,"palo alto, california, united states",united states,linkedin.com/company/tesla-motors,12641,20739,Elon Musk


+ nova coluna size_range: <25000, 25000-50000, 50000-75000, 75000-100000, >100000

In [11]:
new_range = ['75000-100000',
            '75000-100000',
            '75000-100000',
            '>100000',
            '<25000']
dimcompany['size range'] = new_range
dimcompany.head()

,ticker,name,stock_market,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate,CEO
0,AMZN,amazon,NASDAQ,amazon.com,1994.0,internet,75000-100000,"seattle, washington, united states",united states,linkedin.com/company/amazon,93247,161866,Jeff Bezos
1,AAPL,apple,NASDAQ,apple.com,1976.0,consumer electronics,75000-100000,"cupertino, california, united states",united states,linkedin.com/company/apple,90095,179531,Tim Cook
2,GOOGL,google,NASDAQ,google.com,1998.0,internet,75000-100000,"mountain view, california, united states",united states,linkedin.com/company/google,75109,131349,Sundar Pichai
3,MSFT,microsoft,NASDAQ,microsoft.com,1975.0,computer software,>100000,"redmond, washington, united states",united states,linkedin.com/company/microsoft,116196,276983,Satya Nadella
4,TSLA,tesla,NASDAQ,teslamotors.com,2003.0,automotive,<25000,"palo alto, california, united states",united states,linkedin.com/company/tesla-motors,12641,20739,Elon Musk


+ drop de colunas linkedin, employee estimate, total estimate

In [12]:
dimcompany=dimcompany.drop(['linkedin url', 'current employee estimate',
       'total employee estimate'], axis=1)
dimcompany.head()

,ticker,name,stock_market,domain,year founded,industry,size range,locality,country,CEO
0,AMZN,amazon,NASDAQ,amazon.com,1994.0,internet,75000-100000,"seattle, washington, united states",united states,Jeff Bezos
1,AAPL,apple,NASDAQ,apple.com,1976.0,consumer electronics,75000-100000,"cupertino, california, united states",united states,Tim Cook
2,GOOGL,google,NASDAQ,google.com,1998.0,internet,75000-100000,"mountain view, california, united states",united states,Sundar Pichai
3,MSFT,microsoft,NASDAQ,microsoft.com,1975.0,computer software,>100000,"redmond, washington, united states",united states,Satya Nadella
4,TSLA,tesla,NASDAQ,teslamotors.com,2003.0,automotive,<25000,"palo alto, california, united states",united states,Elon Musk


+ separar locality em country, state, city

In [13]:
city=[]
state=[]
country=[]
for row in dimcompany.locality:
  city.append(row.split(', ')[0])
  state.append(row.split(', ')[1])
  country.append(row.split(', ')[2])

dimcompany=dimcompany.drop(['locality','country'], axis=1)
dimcompany['city']=city
dimcompany['state']=state
dimcompany['country']=country
dimcompany

,ticker,name,stock_market,domain,year founded,industry,size range,CEO,city,state,country
0,AMZN,amazon,NASDAQ,amazon.com,1994.0,internet,75000-100000,Jeff Bezos,seattle,washington,united states
1,AAPL,apple,NASDAQ,apple.com,1976.0,consumer electronics,75000-100000,Tim Cook,cupertino,california,united states
2,GOOGL,google,NASDAQ,google.com,1998.0,internet,75000-100000,Sundar Pichai,mountain view,california,united states
3,MSFT,microsoft,NASDAQ,microsoft.com,1975.0,computer software,>100000,Satya Nadella,redmond,washington,united states
4,TSLA,tesla,NASDAQ,teslamotors.com,2003.0,automotive,<25000,Elon Musk,palo alto,california,united states


In [14]:
#Create table dimcompany
SQL = """CREATE TABLE dimcompany (
ticker varchar,
name varchar,
stock_market varchar, 
domain varchar, 
year_founded int, 
industry varchar,
size_range varchar,
CEO varchar,
city varchar,
state varchar,
country varchar,
PRIMARY KEY(ticker));"""

excuteSingleSQLstatement(SQL,host ='localhost',database = database,user ='postgres',password=password)

In [15]:
inserts=[]
nrows=dimcompany.shape[0]
for i in range(nrows):
    line=list(dimcompany.iloc[i])
    ticker="'"+str(line[0])+"'"
    name="'"+str(line[1])+"'" 
    stock_market="'"+str(line[2])+"'"  
    domain="'"+str(line[3])+"'" 
    year_founded=int(line[4])
    industry="'"+str(line[5])+"'" 
    size_range="'"+str(line[6])+"'"
    CEO="'"+str(line[7])+"'"
    city="'"+str(line[8])+"'"
    state="'"+str(line[9])+"'"
    country="'"+str(line[10])+"'"
    
    inserts.append("INSERT INTO dimcompany VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)" % \
                   (ticker,
                    name,
                    stock_market,
                    domain,
                    year_founded,
                    industry,
                    size_range,
                    CEO,
                    city,
                    state,
                    country)
                  )

excuteManySQLstatements(inserts, host="localhost",database=database, user="postgres", password=password)

# Tweets

In [16]:
tweet=pd.read_csv('tweet_final.csv', index_col=0)

c:\users\filip\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [17]:
# drop de métricas da tabela relacional
tweet_dwh=tweet.drop(['post_date', 'comment_num', 'retweet_num', 'like_num'], axis=1)

# dimtweet final
tweet_dwh.head()

,tweet_id,writer,body
0,550441509175443456,VisualStockRSRC,"lx21 made $10,008 on $AAPL -Check it out! htt..."
1,550441672312512512,KeralaGuy77,Insanity of today weirdo massive selling. $aap...
2,550441732014223360,DozenStocks,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...
3,550442977802207232,ShowDreamCar,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...
4,550443807834402816,i_Know_First,Swing Trading: Up To 8.91% Return In 14 Days h...


In [18]:
#Create table dimtweet
SQL = """CREATE TABLE dimtweet (
tweet_id varchar,
writer varchar,
body varchar,
PRIMARY KEY(tweet_id));"""

excuteSingleSQLstatement(SQL,host ='localhost',database = database,user ='postgres',password=password)

In [19]:
inserts=[]
nrows=tweet_dwh.shape[0]
for i in range(nrows):
    line=list(tweet_dwh.iloc[i])
    tweet_id="'"+str(line[0])+"'"
    writer="'"+str(line[1])+"'"
    body=str(line[2]).replace("'","")
    body="'"+body+"'"
    
    inserts.append("INSERT INTO dimtweet VALUES (%s, %s, %s)" % (tweet_id, writer, body))

excuteManySQLstatements(inserts, host="localhost",database=database, user="postgres", password=password)

# Date

In [22]:
SQL="""
create table dimdate(
    pkDate int primary key,
    year int,
    month int,
    day int, 
    quarter char(2),
    semester char(2),
    weekmonth int,
    weekyear int,
    weekday char(16),
    daytype char(16)
)
"""
excuteSingleSQLstatement(SQL,host="localhost",database=database, user="postgres", password=password)

In [23]:
conn = pg.connect(host="localhost",database=database, user="postgres", password=password)
curs = conn.cursor()

# data dos tweets começa em 2015 e vai até 2020 (exclusive)
d=date(year=2015, month=1, day=1)
while d.year<2020:
    d_pkDate    = d.toordinal()
    d_year      = d.year
    d_month     = d.month
    d_day       = d.day
    d_quarter   = getQuarter(d)
    d_semester  = getSemester(d)
    d_weekmonth = getWeekMonth(d)
    d_weekyear  = getWeekYear(d)
    d_weekday   = getWeekDay(d)
    d_daytype   = getDayType(d)
    SQL="insert into dimDate values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    curs.execute(SQL, (d_pkDate,d_year, d_month, d_day,
                       d_quarter, d_semester, d_weekmonth,
                       d_weekyear, d_weekday, d_daytype))
    d+=timedelta(days=1)
curs.close()
conn.commit()
conn.close()

# Facts

Na tabela de factos estão as transações na forma: No dia X foi feito o tweet Y sobre a empresa E, teve L likes, C comments, R retweets, OC diff_openclose, HL diff_highlow

Precisamos de dados da tabela tweet (data, likes, comments e retweets) e stock (volume, diff_openclose e diff_highlow). A tabela de factos será construída a partir da tabela mentions.

In [24]:
data_stock = pd.read_csv("stock_final.csv", index_col = 0)
data_stock.head(5)

,ticker,date,volume,diff_openclose,diff_highlow
0,AAPL,2015-01-02,53204626,2.06,4.09
1,AAPL,2015-01-05,64285491,2.04,3.24
2,AAPL,2015-01-06,65797116,0.28,2.80
3,AAPL,2015-01-07,40105934,-0.55,1.51
4,AAPL,2015-01-08,59364547,-2.66,3.45


In [25]:
tweet.head()

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
0,550441509175443456,VisualStockRSRC,2015-01-01,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,550441672312512512,KeralaGuy77,2015-01-01,Insanity of today weirdo massive selling. $aap...,0,0,0
2,550441732014223360,DozenStocks,2015-01-01,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0
3,550442977802207232,ShowDreamCar,2015-01-01,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1
4,550443807834402816,i_Know_First,2015-01-01,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1


In [26]:
mentions = pd.read_csv("mentions_final.csv", index_col = 0)
mentions=mentions.drop_duplicates()
mentions.head()

c:\users\filip\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,tweet_id,ticker
0,550441509175443456,AAPL
1,550441672312512512,AAPL
2,550441732014223360,AMZN
3,550442977802207232,TSLA
4,550443807834402816,TSLA


In [27]:
facts=pd.merge(mentions, tweet.drop(['writer','body'],axis=1), on=['tweet_id'])
facts.rename(columns={'post_date':'date'}, inplace=True)
facts.head()

,tweet_id,ticker,date,comment_num,retweet_num,like_num
0,550441509175443456,AAPL,2015-01-01,0,0,1
1,550441672312512512,AAPL,2015-01-01,0,0,0
2,550441732014223360,AMZN,2015-01-01,0,0,0
3,550442977802207232,TSLA,2015-01-01,0,0,1
4,550443807834402816,TSLA,2015-01-01,0,0,1


In [28]:
facts=pd.merge(facts, data_stock, on=['ticker','date'])
facts.head()

,tweet_id,ticker,date,comment_num,retweet_num,like_num,volume,diff_openclose,diff_highlow
0,550804137118801921,AMZN,2015-01-02,0,0,1,2788101,4.06,7.79
1,550806376092807168,AMZN,2015-01-02,0,0,1,2788101,4.06,7.79
2,550808188237279232,AMZN,2015-01-02,0,0,1,2788101,4.06,7.79
3,550810934902390785,AMZN,2015-01-02,0,0,1,2788101,4.06,7.79
4,550821049059655681,AMZN,2015-01-02,0,0,1,2788101,4.06,7.79


In [29]:
# transformar data na key de data
keys={}
d=date(year=2015, month=1, day=1)
while d.year<2020:
    d_pkDate    = d.toordinal()
    keys[str(d)]=d_pkDate
    d+=timedelta(days=1)

In [30]:
# tabela de factos final
facts = facts.replace({"date": keys})
facts.head()

,tweet_id,ticker,date,comment_num,retweet_num,like_num,volume,diff_openclose,diff_highlow
0,550804137118801921,AMZN,735600,0,0,1,2788101,4.06,7.79
1,550806376092807168,AMZN,735600,0,0,1,2788101,4.06,7.79
2,550808188237279232,AMZN,735600,0,0,1,2788101,4.06,7.79
3,550810934902390785,AMZN,735600,0,0,1,2788101,4.06,7.79
4,550821049059655681,AMZN,735600,0,0,1,2788101,4.06,7.79


In [31]:
facts.columns=['fk_tweet','fk_company','fk_date', 'comment_num', 'retweet_num', 'like_num',
       'volume', 'diff_openclose', 'diff_highlow']
facts=facts[['fk_date', 'fk_company', 'volume', 'diff_openclose', 'diff_highlow',
            'fk_tweet', 'comment_num', 'retweet_num', 'like_num']]
facts.head(3)

,fk_date,fk_company,volume,diff_openclose,diff_highlow,fk_tweet,comment_num,retweet_num,like_num
0,735600,AMZN,2788101,4.06,7.79,550804137118801921,0,0,1
1,735600,AMZN,2788101,4.06,7.79,550806376092807168,0,0,1
2,735600,AMZN,2788101,4.06,7.79,550808188237279232,0,0,1


In [32]:
facts.to_csv('dwh_facts.csv', index=False)

In [33]:
#Create table tweet_facts

SQL = """CREATE TABLE tweet_facts (
fk_date integer references dimdate(pkDate),
fk_company varchar references dimcompany(ticker), 
volume integer,
diff_openclose integer, 
diff_highlow integer, 
fk_tweet varchar references dimtweet(tweet_id), 
comment_num integer,
retweet_num integer,
like_num integer,
PRIMARY KEY(fk_date, fk_company, fk_tweet));"""

excuteSingleSQLstatement(SQL,host ='localhost',database =database,user ='postgres',password=password)

In [34]:
inserts=[]
nrows=facts.shape[0]
for i in range(nrows):
    line=list(facts.iloc[i])
    fk_date=int(line[0])
    fk_company="'"+str(line[1])+"'"
    volume=int(line[2])
    diff_openclose=float(line[3])
    diff_highlow=float(line[4])
    fk_tweet="'"+str(line[5])+"'"
    comment_num=int(line[6])
    retweet_num=int(line[7])
    like_num=int(line[8])
    
    inserts.append("""INSERT INTO tweet_facts VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""" % (fk_date, 
                                                                                                fk_company, 
                                                                                                volume, 
                                                                                                diff_openclose, 
                                                                                                diff_highlow,
                                                                                                fk_tweet, 
                                                                                                comment_num, 
                                                                                                retweet_num, 
                                                                                                like_num))

excuteManySQLstatements(inserts, host="localhost",database=database, user="postgres", password=password)

In [35]:
facts.shape

(3663402, 9)

In [36]:
mentions.shape

(4288706, 2)

Diferença porque só queremos tweets de dias em que o mercado está aberto (incluir todos e ficar com campos nulos na tabela de factos? incluir todos e usar os ultimos preços conhecidos?)